In [ ]:
import re
import string
import pandas as pd

import nltk
from nltk.corpus import stopwords
# Download Spanish stopwords if not already downloaded
nltk.download('stopwords')

import spacy
import es_core_news_sm
nlp = es_core_news_sm.load()

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\garim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\garim\anaconda3\envs\nlp\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
spanish_stopwords = set(stopwords.words('spanish'))

def remove_stopwords(text, stop_words= spanish_stopwords):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

def lemmatize_words(words):
    doc = nlp(' '.join(words))
    return [token.lemma_ for token in doc]

def preprocess_text(text, remove_stopwords= True, remove_stopwords_func= remove_stopwords):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    # text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()

    if remove_stopwords:
        text = remove_stopwords_func(text)
        
    return text


In [29]:
# Example usage
spanish_text = "¡Hola! Visitas https://example.com para más información. ¿Cuántos años tienes? Tengo 25 años."
cleaned_text = preprocess_text(spanish_text, remove_stopwords= False)
print(cleaned_text)

cleaned_text = remove_stopwords(cleaned_text)
print(cleaned_text)

cleaned_text = " ".join(lemmatize_words(cleaned_text.split()))
print(cleaned_text)

hola visitas para más información cuántos años tienes tengo años
hola visitas información cuántos años años
holar visita información cuántos año año


In [5]:
df = pd.read_csv('data/recetas_limpias.csv')
df.head()

,nombre,url,ingredientes,pasos,pais,duracion,porciones,calorias,categoria,contexto,comensales,tiempo,dificultad,categoria 2,valoracion,votos
0,berenjenas rellenas,https://www.elmueble.com/cocinas/comidas-salud...,"2 berenjenas, 1 pimiento rojo, 1 pimiento amar...",paso 1. lava bien las berenjenas y pártelas po...,españa,45 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"alto en fibra, sin grasas trans, sin sodio o s...",NaN,NaN
1,alcachofas al horno con pico de gallo,https://www.elmueble.com/cocinas/comidas-salud...,"4 alcachofas, 1 limón, 400 g de carne picada (...","paso 1. precalentar el horno a 180?°c. , paso ...",españa,60 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, sin grasa, alto en fibra",NaN,NaN
2,arroz basmati salteado con heura y verduras va...,https://www.elmueble.com/cocinas/comidas-salud...,"200 g de arroz basmati integral, 450 ml de agu...",paso 1. hervir el arroz basmati durante unos 2...,españa,40 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, alto en grasas, bueno fuente...",NaN,NaN
3,tataki de atún,https://www.elmueble.com/cocinas/comidas-salud...,"200 gramos de atún rojo (fresco o congelado), ...",paso 1. cortar en forma rectangular para el ta...,españa,60 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, sin grasa, alto en fibra",NaN,NaN
4,merluza al vapor a la gallega,https://www.elmueble.com/cocinas/comidas-salud...,"700 g de merluza en rodajas, 3 ajos, 1 manojo ...",paso 1. pelar y lavar las patatas. retirar el ...,españa,45 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, sin grasa, alto en fibra",NaN,NaN


In [119]:
# Preprocess the 'pasos' column
train_data = df['pasos'].dropna().apply(preprocess_text).str.split()
train_data = pd.concat([train_data, df['contexto'].dropna().apply(preprocess_text).str.split()], axis=0) # add context sentences
train_data = pd.concat([train_data, df['ingredientes'].dropna().apply(preprocess_text).str.split()], axis=0) # add inhredient list

# Train Word2Vec model
w2v_model = Word2Vec(sentences= train_data, vector_size= 300, window= 5, min_count= 20, workers=4)

In [128]:
w2v_model.wv.most_similar('pollo', topn=10)

[('pavo', 0.6136820316314697),
 ('gallina', 0.5791449546813965),
 ('ave', 0.5639533400535583),
 ('res', 0.5299056768417358),
 ('pato', 0.5087317228317261),
 ('desmenuzada', 0.4685480296611786),
 ('cubo', 0.4501640498638153),
 ('pastilla', 0.447604238986969),
 ('camarón', 0.4317619800567627),
 ('cordero', 0.4316391348838806)]

In [121]:
# find word most similar to a sentence
def most_similar_sentence(model, sentence, topn=10):
    # Preprocess the sentence
    preprocessed_sentence = preprocess_text(sentence)
    # Tokenize the preprocessed sentence
    tokens = preprocessed_sentence.split()
    # Get the vector for the sentence by averaging the word vectors
    sentence_vector = sum(model.wv[word] for word in tokens if word in model.wv) / len(tokens)
    # Find the most similar words to the sentence vector
    similar_words = model.wv.similar_by_vector(sentence_vector, topn=topn)
    return similar_words

most_similar_sentence(w2v_model, "harina de trigo", topn=10)

[('harina', 0.9091925621032715),
 ('trigo', 0.7762859463691711),
 ('maicena', 0.7070057392120361),
 ('levadura', 0.655998945236206),
 ('sémola', 0.6539023518562317),
 ('maizena', 0.6465351581573486),
 ('copos', 0.615618109703064),
 ('fécula', 0.6114292740821838),
 ('harinas', 0.5997712016105652),
 ('sarraceno', 0.5822410583496094)]

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser


# Step 3: Detect frequent bigrams and trigrams. present in at least 5% of the recipes
ingredients_train = df['ingredientes'].dropna().apply(preprocess_text, remove_stopwords= False).str.split()
bigram = Phrases(train_data, min_count= int(df.shape[0] * 0.05), threshold= 20)
trigram = Phrases(bigram[train_data], min_count= int(df.shape[0] * 0.05), threshold=20)

bigram_phraser = Phraser(bigram)
trigram_phraser = Phraser(trigram)

# Step 4: Apply the phrasers to get merged n-grams
ngrammed_corpus = [trigram_phraser[bigram_phraser[doc]] for doc in train_data]


In [ ]:
for ss in ngrammed_corpus:
    for ww in ss:
        if 3 == len(ww.split('_')):
            print(ww)

precalienta_horno_c
precalentar_horno_c
abundante_aceite_oliva
ml_aceite_oliva
minutos_fuego_lento
chorrito_aceite_oliva
abundante_agua_salada
chorrito_aceite_oliva
abundante_agua_salada
retirar_exceso_grasa
abundante_aceite_oliva
siguiendo_instrucciones_paquete
ml_aceite_oliva
pasados_minutos_cocción
pimienta_negra_molida
chorrito_aceite_oliva
microondas_máxima_potencia
espolvoreamos_queso_rallado
fuego_medio_alto
plato_papel_absorbente
retirar_exceso_grasa
dientes_ajo_pelados
dientes_ajo_machacados
sal_pimienta_negra
precalentar_horno_ºc
retirar_exceso_grasa
precalentar_horno_ºc
introducimos_horno_precalentado
sazonamos_sal_pimienta
cucharadas_aceite_oliva
chorrito_vino_blanco
introducimos_horno_precalentado
diente_ajo_picado
carne_pimiento_choricero
cucharada_aceite_oliva
chorrito_vino_blanco
chorrito_vino_blanco
chorrito_aceite_oliva
bajar_fuego_tapar
precalienta_horno_ºc
cucharada_aceite_oliva
diente_ajo_pelado
introducimos_horno_precalentado
abundante_agua_salada
precalentar_horn